In [10]:
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
from matplotlib.ticker import SymmetricalLogLocator
from matplotlib.gridspec import GridSpec
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay
from sklearn.metrics import PrecisionRecallDisplay, RocCurveDisplay, brier_score_loss, roc_auc_score, average_precision_score
from sklearn.utils.estimator_checks import check_estimator
from sklearn import mixture
from sklearn.decomposition import PCA
import scipy

import zarr
from msc import config
from msc.cache_handler import get_samples_df
from msc.estimators import BSLE
from msc.plot_utils import set_size
from msc.time_utils import SEC, MIN, HOUR

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
dataset_id = str(config["dataset_id"])
t_max = config["t_max"]

# load samples_df
samples_df = get_samples_df(dataset_id, with_events=False, with_time_to_event=False)

In [22]:
samples_df

,time,wall_time,embedding
0,39895,2009-07-31 07:16:13+00:00,"[-5.689304351806641, -7.22075080871582, -6.957..."
1,86140,2009-07-31 20:06:58+00:00,"[-4.927209854125977, -6.219898700714111, -5.21..."
2,86295,2009-07-31 20:09:33+00:00,"[-4.493586540222168, -5.52287483215332, -4.884..."
3,92370,2009-07-31 21:50:48+00:00,"[-5.097203254699707, -6.791516780853272, -6.43..."
4,93827,2009-07-31 22:15:05+00:00,"[-4.491633415222168, -5.654514312744141, -4.93..."
...,...,...,...
9621,36907282,2010-10-01 00:12:40+00:00,"[-6.267091274261475, -6.86131477355957, -6.212..."
9622,36919269,2010-10-01 03:32:27+00:00,"[-6.357523918151856, -6.718264102935791, -5.99..."
9623,36921785,2010-10-01 04:14:23+00:00,"[-5.827604293823242, -5.952446937561035, -5.38..."
9624,36921825,2010-10-01 04:15:03+00:00,"[-4.460057735443115, -3.253157138824463, -4.48..."


In [23]:
samples_df.loc[0, 'embedding'].shape

(274,)

In [11]:
cache_path = f"{config['path']['data']}/cache.zarr"
cache_zarr = zarr.open(cache_path, "r")


In [12]:
ds_zarr = cache_zarr[f"{dataset_id}"]


In [20]:
ds_zarr['39895']['embedding']

<zarr.core.Array '/I004_A0003_D001/39895/embedding' (277,) float64 read-only>